In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/lexihenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
yrs=np.arange(1980,2024,1)

if 1==1:
    ds=xr.open_dataset(dir_data+'ar_vars_geomeans_era5_MAX.nc')
    ivt_concat_era5_fixed_poleward=ds['ivt_concat_era5_fixed_poleward'].values
    iwv_concat_era5_fixed_poleward=ds['iwv_concat_era5_fixed_poleward'].values
    s_concat_era5_fixed_poleward=ds['s_concat_era5_fixed_poleward'].values
    mfc_concat_era5_fixed_poleward=ds['mfc_concat_era5_fixed_poleward'].values
    
    ivt_concat_era5_fixed=ds['ivt_concat_era5_fixed'].values
    iwv_concat_era5_fixed=ds['iwv_concat_era5_fixed'].values
    s_concat_era5_fixed=ds['s_concat_era5_fixed'].values
    mfc_concat_era5_fixed=ds['mfc_concat_era5_fixed'].values
    
    ivt_concat_era5_polar=ds['ivt_concat_era5_variable'].values
    iwv_concat_era5_polar=ds['iwv_concat_era5_variable'].values
    s_concat_era5_polar=ds['s_concat_era5_variable'].values
    mfc_concat_era5_polar=ds['mfc_concat_era5_variable'].values
    
for i in range(29,len(yrs)):
    year=yrs[i]
    print(year)
    
    ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
    ivt=ds['ivt']
    
    ds=xr.open_dataset(dir2+'era_5_iwvinterp_'+str(year)+'.nc')
    iwv=ds['iwv']
    
    ds=xr.open_dataset(dir2+'era_5_u850interp_'+str(year)+'.nc')
    u=ds['u']
    ds=xr.open_dataset(dir2+'era_5_v850interp_'+str(year)+'.nc')
    v=ds['v']
    s=np.sqrt(u**2+v**2)
    
    ds=xr.open_dataset(dir2+'era_5_mfcinterp_'+str(year)+'.nc')
    mfc=ds['mfc']*-1
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.variable.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    ivt['lon']=labeled.lon.values
    iwv['lon']=labeled.lon.values
    s['lon']=labeled.lon.values
    mfc['lon']=labeled.lon.values
        
    ivt['lat']=labeled.lat.values
    iwv['lat']=labeled.lat.values
    s['lat']=labeled.lat.values
    mfc['lat']=labeled.lat.values

    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s_sel=s.where(labeled>0)
    s_mean=s_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    mfc_sel=mfc.where(labeled>0)
    mfc_mean=mfc_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    
    if i==0:
        ivt_concat_era5_polar=ivt_mean
        iwv_concat_era5_polar=iwv_mean
        s_concat_era5_polar=s_mean
        mfc_concat_era5_polar=mfc_mean
    else:
        #ivt_concat_era5_polar=xr.concat([ivt_concat_era5_polar,ivt_mean],dim='time')
        #iwv_concat_era5_polar=xr.concat([iwv_concat_era5_polar,iwv_mean],dim='time')
        #s_concat_era5_polar=xr.concat([s_concat_era5_polar,s_mean],dim='time')
        #mfc_concat_era5_polar=xr.concat([mfc_concat_era5_polar,mfc_mean],dim='time')
        
        ivt_concat_era5_polar=np.concatenate((ivt_concat_era5_polar,ivt_mean),axis=0)
        iwv_concat_era5_polar=np.concatenate((iwv_concat_era5_polar,iwv_mean),axis=0)
        s_concat_era5_polar=np.concatenate((s_concat_era5_polar,s_mean),axis=0)
        mfc_concat_era5_polar=np.concatenate((mfc_concat_era5_polar,mfc_mean),axis=0)
    
    ds=xr.open_dataset(dir2+'era5.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    ivt['lon']=labeled.lon.values
    iwv['lon']=labeled.lon.values
    s['lon']=labeled.lon.values
    mfc['lon']=labeled.lon.values
        
    ivt['lat']=labeled.lat.values
    iwv['lat']=labeled.lat.values
    s['lat']=labeled.lat.values
    mfc['lat']=labeled.lat.values

    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s_sel=s.where(labeled>0)
    s_mean=s_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    mfc_sel=mfc.where(labeled>0)
    mfc_mean=mfc_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    
    if i==0:
        ivt_concat_era5_fixed=ivt_mean
        iwv_concat_era5_fixed=iwv_mean
        s_concat_era5_fixed=s_mean
        mfc_concat_era5_fixed=mfc_mean
    else:
        #ivt_concat_era5_fixed=xr.concat([ivt_concat_era5_fixed,ivt_mean],dim='time')
        #iwv_concat_era5_fixed=xr.concat([iwv_concat_era5_fixed,iwv_mean],dim='time')
        #s_concat_era5_fixed=xr.concat([s_concat_era5_fixed,s_mean],dim='time')
        #mfc_concat_era5_fixed=xr.concat([mfc_concat_era5_fixed,mfc_mean],dim='time')
        
        ivt_concat_era5_fixed=np.concatenate((ivt_concat_era5_fixed,ivt_mean),axis=0)
        iwv_concat_era5_fixed=np.concatenate((iwv_concat_era5_fixed,iwv_mean),axis=0)
        s_concat_era5_fixed=np.concatenate((s_concat_era5_fixed,s_mean),axis=0)
        mfc_concat_era5_fixed=np.concatenate((mfc_concat_era5_fixed,mfc_mean),axis=0)
    

    ds=xr.open_dataset(dir2+'era5.ar.labels.model.poleward.'+str(year)+'.nc')
    labeled=ds['ar_labeled']
    
    ivt['lon']=labeled.lon.values
    iwv['lon']=labeled.lon.values
    s['lon']=labeled.lon.values
    mfc['lon']=labeled.lon.values
        
    ivt['lat']=labeled.lat.values
    iwv['lat']=labeled.lat.values
    s['lat']=labeled.lat.values
    mfc['lat']=labeled.lat.values

    iwv_sel=iwv.where(labeled>0)
    iwv_mean=iwv_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    ivt_sel=ivt.where(labeled>0)
    ivt_mean=ivt_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s_sel=s.where(labeled>0)
    s_mean=s_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    mfc_sel=mfc.where(labeled>0)
    mfc_mean=mfc_sel.max(dim='time',skipna=True).values[np.newaxis,:,:]
    
    if i==0:
        ivt_concat_era5_fixed_poleward=ivt_mean
        iwv_concat_era5_fixed_poleward=iwv_mean
        s_concat_era5_fixed_poleward=s_mean
        mfc_concat_era5_fixed_poleward=mfc_mean
    else:
        #ivt_concat_era5_fixed_poleward=xr.concat([ivt_concat_era5_fixed_poleward,ivt_mean],dim='time')
        #iwv_concat_era5_fixed_poleward=xr.concat([iwv_concat_era5_fixed_poleward,iwv_mean],dim='time')
        #s_concat_era5_fixed_poleward=xr.concat([s_concat_era5_fixed_poleward,s_mean],dim='time')
        #mfc_concat_era5_fixed_poleward=xr.concat([mfc_concat_era5_fixed_poleward,mfc_mean],dim='time')
        
        ivt_concat_era5_fixed_poleward=np.concatenate((ivt_concat_era5_fixed_poleward,ivt_mean),axis=0)
        iwv_concat_era5_fixed_poleward=np.concatenate((iwv_concat_era5_fixed_poleward,iwv_mean),axis=0)
        s_concat_era5_fixed_poleward=np.concatenate((s_concat_era5_fixed_poleward,s_mean),axis=0)
        mfc_concat_era5_fixed_poleward=np.concatenate((mfc_concat_era5_fixed_poleward,mfc_mean),axis=0)

In [ ]:
dk=xr.Dataset()
dk['ivt_concat_era5_variable']=(('time','lat','lon'),ivt_concat_era5_polar)
dk['iwv_concat_era5_variable']=(('time','lat','lon'),iwv_concat_era5_polar)
dk['s_concat_era5_variable']=(('time','lat','lon'),s_concat_era5_polar)
dk['mfc_concat_era5_variable']=(('time','lat','lon'),mfc_concat_era5_polar)

dk['ivt_concat_era5_fixed']=(('time','lat','lon'),ivt_concat_era5_fixed)
dk['iwv_concat_era5_fixed']=(('time','lat','lon'),iwv_concat_era5_fixed)
dk['s_concat_era5_fixed']=(('time','lat','lon'),s_concat_era5_fixed)
dk['mfc_concat_era5_fixed']=(('time','lat','lon'),mfc_concat_era5_fixed)

dk['ivt_concat_era5_fixed_poleward']=(('time','lat','lon'),ivt_concat_era5_fixed_poleward)
dk['iwv_concat_era5_fixed_poleward']=(('time','lat','lon'),iwv_concat_era5_fixed_poleward)
dk['s_concat_era5_fixed_poleward']=(('time','lat','lon'),s_concat_era5_fixed_poleward)
dk['mfc_concat_era5_fixed_poleward']=(('time','lat','lon'),mfc_concat_era5_fixed_poleward)

dk.coords['time']=np.arange(1980,2024)
dk.coords['lat']=ivt.lat.values
dk.coords['lon']=ivt.lon.values
dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'
dk['time'].attrs["units"]='years'
print(dk)
try:
    os.remove(dir_data+'ar_vars_geomeans_era5_MAX.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'ar_vars_geomeans_era5_MAX.nc',mode='w',format='NETCDF4')

In [ ]:
if 1==1:
    ds=xr.open_dataset(dir_data+'ar_vars_geomeans_era5_MAX.nc')
    ivt_concat_era5_fixed_poleward=ds['ivt_concat_era5_fixed_poleward'].values
    iwv_concat_era5_fixed_poleward=ds['iwv_concat_era5_fixed_poleward'].values
    s_concat_era5_fixed_poleward=ds['s_concat_era5_fixed_poleward'].values
    mfc_concat_era5_fixed_poleward=ds['mfc_concat_era5_fixed_poleward'].values
    
    ds=xr.open_dataset(dir_data+'ar_vars_geomeans_era5_MAX.nc')
    ivt_concat_era5_fixed=ds['ivt_concat_era5_fixed'].values
    iwv_concat_era5_fixed=ds['iwv_concat_era5_fixed'].values
    s_concat_era5_fixed=ds['s_concat_era5_fixed'].values
    mfc_concat_era5_fixed=ds['mfc_concat_era5_fixed'].values
    
    ds=xr.open_dataset(dir_data+'ar_vars_geomeans_era5_MAX.nc')
    ivt_concat_era5_polar=ds['ivt_concat_era5_variable'].values
    iwv_concat_era5_polar=ds['iwv_concat_era5_variable'].values
    s_concat_era5_polar=ds['s_concat_era5_variable'].values
    mfc_concat_era5_polar=ds['mfc_concat_era5_variable'].values